In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 198 kB 15.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=035d5547845dc42e788d272d3b46f4df859758219ebd866548d3318cccd958db
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
input_data = '/content/drive/MyDrive/CLC/data/used_cars_data.csv'

In [ ]:
sc= SparkContext()
sqlContext = SQLContext(sc)


In [ ]:
car_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(input_data)
car_df.show(3)

+-----------------+------------+----+----------+----------+---------------+-----+--------+-----------------+--------------------+------------+----------+--------------------+----------------+-------------------+-----------+--------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+-------+--------------------+----------+--------------+-----+------------+------+------+---------+--------+--------+-----------+-------------+----------+---------+--------------------+--------------------+----------+---------------+-------+---------------+-----------+------------------+-------+-------+--------------+-------------+------+-------------------+-----------+--------------------+------------+--------------------+------+------------+-----------------------+------------+--------------------+---------+-------+----+
|              vin|back_legroom| bed|bed_height|bed_length|      body_type|cabin|    city|city_fuel_economy|combine_fuel_econom

In [ ]:
car_df.printSchema()

root
 |-- vin: string (nullable = true)
 |-- back_legroom: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- bed_height: string (nullable = true)
 |-- bed_length: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: string (nullable = true)
 |-- combine_fuel_economy: string (nullable = true)
 |-- daysonmarket: string (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- description: string (nullable = true)
 |-- engine_cylinders: string (nullable = true)
 |-- engine_displacement: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- exterior_color: string (nullable = true)
 |-- fleet: string (nullable = true)
 |-- frame_damaged: string (nullable = true)
 |-- franchise_dealer: string (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: string (nullable = true)
 |-- fuel_tank_volume: string (nullable = tr

In [ ]:
car_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
vin,3000507,Infinity,NaN,Heated Windshield Washer nozzles,type: SUV
back_legroom,2840909,None,None,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
bed,19679,4081.25,1653.3677751788923,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,Volkswagen
bed_height,429196,5166.666666666667,1474.2229591663988,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
...,...,...,...,...,...
wheel_system,2853203,1.2678890726677366E7,5.764808861529345E7,,~PUSH BUTTON START~
wheel_system_display,2848662,1.3396662595330492E7,5.918315832965848E7,,~~~ Call (502) 695-9050 to check availability ...
wheelbase,2828164,1.4679855323887102E7,6.178457258018881E7,,~PUSH BUTTON START~
width,2830498,1.384501351400201E7,6.009541186023207E7,,~GM CERTIFIED


In [ ]:
car_df = car_df.withColumn("back_legroom", f.regexp_replace(f.col("back_legroom"), " in", ""))
car_df = car_df.withColumn("height", f.regexp_replace(f.col("height"), " in", ""))
car_df = car_df.withColumn("length", f.regexp_replace(f.col("length"), " in", ""))
car_df = car_df.withColumn("maximum_seating", f.regexp_replace(f.col("maximum_seating"), " seats", ""))
car_df = car_df.withColumn("front_legroom", f.regexp_replace(f.col("front_legroom"), " in", ""))
car_df = car_df.withColumn("fuel_tank_volume", f.regexp_replace(f.col("fuel_tank_volume"), " gal", ""))

In [ ]:
car_df = car_df.withColumn("back_legroom", car_df["back_legroom"].cast(IntegerType()))
car_df = car_df.withColumn("city_fuel_economy", car_df["city_fuel_economy"].cast(IntegerType()))
car_df = car_df.withColumn("daysonmarket", car_df["daysonmarket"].cast(IntegerType()))
car_df = car_df.withColumn("engine_displacement", car_df["engine_displacement"].cast(IntegerType()))
car_df = car_df.withColumn("front_legroom", car_df["front_legroom"].cast(IntegerType()))
car_df = car_df.withColumn("fuel_tank_volume", car_df["fuel_tank_volume"].cast(IntegerType()))
car_df = car_df.withColumn("height", car_df["height"].cast(IntegerType()))
car_df = car_df.withColumn("horsepower", car_df["horsepower"].cast(IntegerType()))
car_df = car_df.withColumn("length", car_df["length"].cast(IntegerType()))
car_df = car_df.withColumn("maximum_seating", car_df["maximum_seating"].cast(IntegerType()))
car_df = car_df.withColumn("seller_rating", car_df["seller_rating"].cast(IntegerType()))
car_df = car_df.withColumn("year", car_df["year"].cast(IntegerType()))
car_df = car_df.withColumn("price", car_df["price"].cast(IntegerType()))
car_df = car_df.withColumn("mileage", car_df["mileage"].cast(IntegerType()))
car_df = car_df.withColumn("is_new", car_df["is_new"].cast(IntegerType()))

In [ ]:
car_df.show(50)

+-----------------+------------+----+----------+----------+---------------+-----+---------+-----------------+--------------------+------------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------+----------------+--------------------+--------------------+------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+-------+--------------------+--------------------+--------------------+-----+-------------------+------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------

In [ ]:
car_df = car_df.na.drop(subset=['back_legroom', 
                       #'franchise_make', 'city', 'engine_cylinders', 'fuel_type', 'interior_color', 'is_new', 
                       'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 
                       'fuel_tank_volume', 'height', 'horsepower', 'length', 'maximum_seating', 
                       'mileage', 'seller_rating', 'year'])

In [ ]:
car_df.printSchema()

root
 |-- vin: string (nullable = true)
 |-- back_legroom: integer (nullable = true)
 |-- bed: string (nullable = true)
 |-- bed_height: string (nullable = true)
 |-- bed_length: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: integer (nullable = true)
 |-- combine_fuel_economy: string (nullable = true)
 |-- daysonmarket: integer (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- description: string (nullable = true)
 |-- engine_cylinders: string (nullable = true)
 |-- engine_displacement: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- exterior_color: string (nullable = true)
 |-- fleet: string (nullable = true)
 |-- frame_damaged: string (nullable = true)
 |-- franchise_dealer: string (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: integer (nullable = true)
 |-- fuel_tank_volume: integer (nullabl

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['back_legroom', 
                                               #'franchise_make', 'city', 'engine_cylinders', 'fuel_type', 'interior_color', 'is_new', 
                                               'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 
                                               'fuel_tank_volume', 'height', 'horsepower', 'length', 'maximum_seating', 
                                               'mileage', 'seller_rating', 'year'], outputCol = 'features')
vcar_df = vectorAssembler.transform(car_df)
vcar_df = vcar_df.select(['features', 'price'])
#vcar_df.show(3)

In [ ]:
vcar_df.show(3)

+--------------------+-----+
|            features|price|
+--------------------+-----+
|[38.0,27.0,55.0,1...|14639|
|[38.0,18.0,27.0,3...|23723|
|[36.0,15.0,27.0,3...|22422|
+--------------------+-----+
only showing top 3 rows



In [ ]:
splits = vcar_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

### 1. Linear Regression

In [ ]:
%%time
lr = LinearRegression(featuresCol = 'features', labelCol='price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [93.13205861845567,230.46391133491335,5.210794950854881,-1.555050357070456,-334.35148339920994,302.01124411867187,159.69534185394915,122.89280746606613,-75.2084243196756,-42.080358112212465,-0.10498347181095287,887.5357554477868,912.166258892417]
Intercept: -1833756.7058709606
CPU times: user 2.42 s, sys: 251 ms, total: 2.67 s
Wall time: 9min 12s


In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 9387.767665
r2: 0.656307


In [ ]:
lr_model.save('/content/drive/MyDrive/CLC/model')

In [ ]:
loadModel = LinearRegressionModel.load("/content/drive/MyDrive/CLC/model")

In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","price","features").show(5)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+-----+--------------------+
|         prediction|price|            features|
+-------------------+-----+--------------------+
| 469.07634108234197| 5999|[0.0,12.0,20.0,47...|
|  8384.073076879838| 4399|[0.0,12.0,70.0,54...|
| 11263.341331365285| 4895|[0.0,12.0,200.0,5...|
|-340.83528415323235| 3999|[0.0,13.0,61.0,46...|
| 10374.831838193815| 5995|[0.0,13.0,95.0,54...|
+-------------------+-----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.6629


In [ ]:
lr_predictions = loadModel.transform(test_df)
lr_predictions.select("prediction","price","features").show(5)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+-----+--------------------+
|         prediction|price|            features|
+-------------------+-----+--------------------+
| 469.07634108234197| 5999|[0.0,12.0,20.0,47...|
|  8384.073076879838| 4399|[0.0,12.0,70.0,54...|
| 11263.341331365285| 4895|[0.0,12.0,200.0,5...|
|-340.83528415323235| 3999|[0.0,13.0,61.0,46...|
| 10374.831838193815| 5995|[0.0,13.0,95.0,54...|
+-------------------+-----+--------------------+
only showing top 5 rows



KeyboardInterrupt: ignored

In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 8686.85


### Test on new data

In [ ]:
import pandas as pd

new_data = pd.read_csv("/content/drive/MyDrive/CLC/data/used_cars_data.csv", nrows=1000, 
                       usecols=['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 
                                'fuel_tank_volume', 'height', 'horsepower', 'length', 'maximum_seating', 
                                'mileage', 'seller_rating', 'year', 'price'])
new_data.dropna(inplace=True)

In [ ]:
new_data 

,back_legroom,city_fuel_economy,daysonmarket,engine_displacement,front_legroom,fuel_tank_volume,height,horsepower,length,maximum_seating,mileage,price,seller_rating,year
38,38.1 in,27.0,55,1500.0,42 in,15.8 gal,57.6 in,160.0,193.8 in,5 seats,42394.0,14639.0,3.447761,2018
40,27.3 in,18.0,36,3500.0,45.4 in,17.4 gal,55.1 in,311.0,184.8 in,4 seats,62251.0,32000.0,2.800000,2018
41,38.4 in,18.0,27,3600.0,41 in,19.4 gal,70.7 in,310.0,204.3 in,8 seats,36410.0,23723.0,3.447761,2018
45,36.8 in,15.0,27,3600.0,41.3 in,22 gal,69.9 in,281.0,203.7 in,8 seats,36055.0,22422.0,3.447761,2017
47,38.6 in,18.0,24,3600.0,40.3 in,24.6 gal,69.3 in,295.0,189.8 in,5 seats,25745.0,29424.0,3.447761,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,38.6 in,18.0,5,3600.0,40.3 in,24.6 gal,69.3 in,295.0,189.8 in,5 seats,1.0,36338.0,4.377778,2020
991,38 in,29.0,41,2000.0,41.4 in,13.2 gal,64.2 in,146.0,172 in,5 seats,11.0,23585.0,3.647059,2021
992,38.8 in,28.0,55,2000.0,41.1 in,14.3 gal,63 in,147.0,165.2 in,5 seats,12.0,23505.0,3.647059,2021
993,30.3 in,25.0,13,1800.0,42.6 in,13.2 gal,55 in,140.0,175.5 in,5 seats,181465.0,3750.0,3.577778,2010


In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .getOrCreate()

In [ ]:
sparkDF=spark.createDataFrame(new_data.astype(str))

In [ ]:
sparkDF = sparkDF.withColumn("back_legroom", f.regexp_replace(f.col("back_legroom"), " in", ""))
sparkDF = sparkDF.withColumn("height", f.regexp_replace(f.col("height"), " in", ""))
sparkDF = sparkDF.withColumn("length", f.regexp_replace(f.col("length"), " in", ""))
sparkDF = sparkDF.withColumn("maximum_seating", f.regexp_replace(f.col("maximum_seating"), " seats", ""))
sparkDF = sparkDF.withColumn("front_legroom", f.regexp_replace(f.col("front_legroom"), " in", ""))
sparkDF = sparkDF.withColumn("fuel_tank_volume", f.regexp_replace(f.col("fuel_tank_volume"), " gal", ""))

In [ ]:
sparkDF = sparkDF.withColumn("back_legroom", sparkDF["back_legroom"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("city_fuel_economy", sparkDF["city_fuel_economy"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("daysonmarket", sparkDF["daysonmarket"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("engine_displacement", sparkDF["engine_displacement"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("front_legroom", sparkDF["front_legroom"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("fuel_tank_volume", sparkDF["fuel_tank_volume"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("height", sparkDF["height"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("horsepower", sparkDF["horsepower"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("length", sparkDF["length"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("maximum_seating", sparkDF["maximum_seating"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("seller_rating", sparkDF["seller_rating"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("year", sparkDF["year"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("price", sparkDF["price"].cast(IntegerType()))
sparkDF = sparkDF.withColumn("mileage", sparkDF["mileage"].cast(IntegerType()))

In [ ]:
sparkDF.show(5)

+------------+-----------------+------------+-------------------+-------------+----------------+------+----------+------+---------------+-------+-----+-------------+----+
|back_legroom|city_fuel_economy|daysonmarket|engine_displacement|front_legroom|fuel_tank_volume|height|horsepower|length|maximum_seating|mileage|price|seller_rating|year|
+------------+-----------------+------------+-------------------+-------------+----------------+------+----------+------+---------------+-------+-----+-------------+----+
|          38|               27|          55|               1500|           42|              15|    57|       160|   193|              5|  42394|14639|            3|2018|
|          27|               18|          36|               3500|           45|              17|    55|       311|   184|              4|  62251|32000|            2|2018|
|          38|               18|          27|               3600|           41|              19|    70|       310|   204|              8|  36410|

In [ ]:
sparkDF.toPandas().to_csv('/content/drive/MyDrive/CLC/data/test.csv')

In [ ]:
import pandas as pd
new_data = pd.read_csv("/content/drive/MyDrive/CLC/data/test.csv")

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['back_legroom', 
                                               #'franchise_make', 'city', 'engine_cylinders', 'fuel_type', 'interior_color', 'is_new', 
                                               'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 
                                               'fuel_tank_volume', 'height', 'horsepower', 'length', 'maximum_seating', 
                                               'mileage', 'seller_rating', 'year'], outputCol = 'features')
vcar_df = vectorAssembler.transform(sparkDF)
#vcar_df = vcar_df.select(['features', 'price'])

AttributeError: ignored

In [ ]:
loadModel = LinearRegressionModel.load("/content/drive/MyDrive/CLC/model")

In [ ]:
lr_predictions = loadModel.transform(vcar_df)
lr_predictions = lr_predictions.select("prediction",
                      #"price",
                      "features")

In [ ]:
lr_predictions.select('prediction').show(5)

+------------------+
|        prediction|
+------------------+
|17427.374874186702|
| 26636.66364334454|
| 33721.74726729654|
|29238.961087327916|
| 35932.04094431852|
+------------------+
only showing top 5 rows



### 2. Decision Tree

In [ ]:
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'price')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 7495.51


### 3. Gradient Boost

In [ ]:
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'price', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'price', 'features').show(5)

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
|12889.507348475327| 4399|[0.0,12.0,70.0,54...|
| 12287.89393300239| 5999|[0.0,13.0,4.0,530...|
|11987.593865374238| 4999|[0.0,13.0,60.0,57...|
|12387.909781446891| 5995|[0.0,13.0,95.0,54...|
|20460.777444931253|13888|[0.0,14.0,13.0,57...|
+------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 6364.64


In [19]:
weights = {'weights':[-1808474.8526028227, 9.98875287e+01,  2.27108658e+02,  5.00486434e+00, -1.53351270e+00,
       -3.38395366e+02,  3.01219000e+02,  1.57727901e+02,  1.22752017e+02,
       -7.39318301e+01, -4.43026102e+01, -1.06698619e-01,  8.84177912e+02,
        8.99615600e+02]}

In [1]:
import json

In [12]:
import smart_open
model_f = smart_open.open('/content/weights.txt', 'r')

In [14]:
model_f.readline()

''

In [18]:
model_f.readline()

[]

In [7]:
import json
model_f = model_f.readline()
json_acceptable_string = model_f.replace("'", "\"")
d = json.loads(json_acceptable_string)

JSONDecodeError: ignored

In [20]:
import numpy as np
w = np.array(weights['weights'])

In [21]:
w

array([-1.80847485e+06,  9.98875287e+01,  2.27108658e+02,  5.00486434e+00,
       -1.53351270e+00, -3.38395366e+02,  3.01219000e+02,  1.57727901e+02,
        1.22752017e+02, -7.39318301e+01, -4.43026102e+01, -1.06698619e-01,
        8.84177912e+02,  8.99615600e+02])

In [22]:
w[1:]

array([ 9.98875287e+01,  2.27108658e+02,  5.00486434e+00, -1.53351270e+00,
       -3.38395366e+02,  3.01219000e+02,  1.57727901e+02,  1.22752017e+02,
       -7.39318301e+01, -4.43026102e+01, -1.06698619e-01,  8.84177912e+02,
        8.99615600e+02])

In [23]:
import pandas as pd

df = pd.read_csv('/content/test.csv', index_col=0)

In [24]:
df_np = np.array(df)

In [25]:
df_np[0]

array([3.8000e+01, 2.7000e+01, 5.5000e+01, 1.5000e+03, 4.2000e+01,
       1.5000e+01, 5.7000e+01, 1.6000e+02, 1.9300e+02, 5.0000e+00,
       4.2394e+04, 3.0000e+00, 2.0180e+03])

In [26]:
w[1:]

array([ 9.98875287e+01,  2.27108658e+02,  5.00486434e+00, -1.53351270e+00,
       -3.38395366e+02,  3.01219000e+02,  1.57727901e+02,  1.22752017e+02,
       -7.39318301e+01, -4.43026102e+01, -1.06698619e-01,  8.84177912e+02,
        8.99615600e+02])

In [27]:
w[0]+np.sum(df_np[0]*w[1:])

17427.375469291117

In [28]:
x = pd.Series(w[0]+np.matmul(w[1:], np.transpose(df_np)))

In [30]:
x.to_dict()

{0: 17427.375469291117,
 1: 26636.66426774813,
 2: 33721.74787916709,
 3: 29238.96169361216,
 4: 35932.04154988215,
 5: 19703.184810044244,
 6: 21133.63931407011,
 7: 29484.75003470434,
 8: 33671.67333493312,
 9: 33836.95131844329,
 10: 17129.217729191296,
 11: 26462.22444759519,
 12: 34718.501508792164,
 13: 19124.397527869325,
 14: 22261.929908890277,
 15: 35388.265187449055,
 16: 33398.1785684342,
 17: 33600.18760975427,
 18: 15532.00942622032,
 19: 37691.14471929311,
 20: 20762.547729114303,
 21: 4995.76315021445,
 22: 33523.384021670325,
 23: 25186.576760078315,
 24: 28127.11680654832,
 25: 16257.847188471118,
 26: 16863.86797511927,
 27: 15442.571770078037,
 28: 32139.848168658325,
 29: nan,
 30: 27044.09935760824,
 31: 76634.92220938904,
 32: 35027.78203747817,
 33: 19521.733457198367,
 34: 35684.71232052334,
 35: 52091.18264433206,
 36: 21735.29272963223,
 37: 22673.0512945971,
 38: nan,
 39: 41338.67888911534,
 40: 18937.20619148016,
 41: 30593.094840608304,
 42: 27303.0848866

In [33]:
x = list(w[0]+np.matmul(w[1:], np.transpose(df_np)))

In [36]:
x = list(w[0]+np.matmul(w[1:], np.transpose(df_np)))
cars_list = ['car_'+ str(i)  for i in range(1, len(x)+1)] 

In [40]:
df_res = pd.DataFrame(columns=['Car', 'Prediction'])
df_res.Car=cars_list
df_res.Prediction=x

In [ ]:
dt.DataTable(
            rows=df.to_dict('records'),
            columns=df.columns,
            row_selectable=True,
            filterable=True,
            sortable=True,
            selected_row_indices=list(df.index),  # all rows selected by default
            id='2'
     )